# 🦠 Dashboard IMUNOPREVINIVEIS - Versão Jupyter Notebook

Esta é uma versão adaptada do dashboard Streamlit para funcionar em Jupyter Notebook.

## 📋 Funcionalidades Disponíveis:
- 🏠 Visão Geral 2024
- 📊 Análise de Casos
- 🦠 Análise de Sorogrupos
- 🔬 Análise de Etiologia
- 💉 Análise de Imunização
- 🗺️ Análise Regional
- 👶 Faixa Etária
- 🔬 Análises Avançadas
- 🦠 Análise Epidemiológica
- ⚡ Taxa de Ataque
- 🧮 Modelagem SIR
- 🔍 Exploração Livre
- 📋 Relatórios
- ⚙️ Expositivo Técnico


In [ ]:
# Importações necessárias
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
from scipy import stats
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import ipywidgets as widgets
from IPython.display import display, clear_output
import sys
import os

# Configurar warnings
warnings.filterwarnings('ignore')

# Configurar matplotlib para inline
%matplotlib inline

# Configurar plotly para notebook
import plotly.io as pio
pio.renderers.default = "notebook"

print("📊 Bibliotecas carregadas com sucesso!")


In [ ]:
# Função para carregar dados (adaptada do dashboard_completo_v2.py)
def load_all_data():
    """Carrega todos os dados processados"""
    
    print("📊 Carregando todos os dados processados...")
    
    try:
        # Dados básicos
        casos_consolidados = pd.read_csv('TABELAS/casos_consolidados_2017_2024.csv')
        sorogrupos_consolidados = pd.read_csv('TABELAS/sorogrupos_consolidados_2007_2024.csv')
        etiologias_consolidadas = pd.read_csv('TABELAS/etiologias_consolidadas_2007_2024.csv')
        
        # Dados de imunização
        imunizacao_ano = pd.read_csv('TABELAS/imunizacao_por_ano.csv')
        imunizacao_faixa_etaria = pd.read_csv('TABELAS/imunizacao_por_faixa_etaria.csv')
        imunizacao_sorogrupo = pd.read_csv('TABELAS/imunizacao_por_sorogrupo.csv')
        imunizacao_uf = pd.read_csv('TABELAS/imunizacao_por_uf.csv')
        
        # Dados de letalidade e casos
        letalidade_etiologia = pd.read_csv('TABELAS/letalidade_etiologia_2007_2020.csv')
        casos_2017_2022 = pd.read_csv('TABELAS/casos_notificados_2017_2022.csv')
        dados_gerais_2024 = pd.read_csv('TABELAS/dados_gerais_2024.csv')
        bacterianas_2024 = pd.read_csv('TABELAS/bacterianas_2024.csv')
        etiologia_2024 = pd.read_csv('TABELAS/etiologia_2024.csv')
        sorogrupos_2024 = pd.read_csv('TABELAS/sorogrupos_2024.csv')
        
        # Dados de imunização 2023-2025
        try:
            imunizacao_2023_2025 = pd.read_csv('TABELAS/imunobiologicosem2023a2025.csv')
        except Exception:
            imunizacao_2023_2025 = None
        
        # Dados de imunização processados
        try:
            imunizacao_processada = pd.read_csv('data/processed/dados_imunizacao_processados.csv')
        except Exception:
            imunizacao_processada = None
        
        # Dados de hospitalização SIH
        sih_meningite = pd.read_csv('TABELAS/sih_meningite_long.csv')
        
        print("✅ Dados carregados com sucesso!")
        
        return {
            'casos_consolidados': casos_consolidados,
            'sorogrupos_consolidados': sorogrupos_consolidados,
            'etiologias_consolidadas': etiologias_consolidadas,
            'imunizacao_ano': imunizacao_ano,
            'imunizacao_faixa_etaria': imunizacao_faixa_etaria,
            'imunizacao_sorogrupo': imunizacao_sorogrupo,
            'imunizacao_uf': imunizacao_uf,
            'imunizacao_2023_2025': imunizacao_2023_2025,
            'imunizacao_processada': imunizacao_processada,
            'letalidade_etiologia': letalidade_etiologia,
            'casos_2017_2022': casos_2017_2022,
            'dados_gerais_2024': dados_gerais_2024,
            'bacterianas_2024': bacterianas_2024,
            'etiologia_2024': etiologia_2024,
            'sorogrupos_2024': sorogrupos_2024,
            'sih_meningite': sih_meningite
        }
        
    except Exception as e:
        print(f"❌ Erro ao carregar dados: {str(e)}")
        return None

# Carregar dados
dados = load_all_data()


In [ ]:
# Widget de navegação principal
def create_navigation_widget():
    """Cria o widget de navegação principal"""
    
    opcoes = [
        "🏠 Visão Geral 2024",
        "📊 Análise de Casos",
        "🦠 Análise de Sorogrupos",
        "🔬 Análise de Etiologia",
        "💉 Análise de Imunização",
        "🗺️ Análise Regional",
        "👶 Faixa Etária",
        "🔬 Análises Avançadas",
        "🦠 Análise Epidemiológica",
        "⚡ Taxa de Ataque",
        "🧮 Modelagem SIR",
        "🔍 Exploração Livre",
        "📋 Relatórios",
        "⚙️ Expositivo Técnico"
    ]
    
    dropdown = widgets.Dropdown(
        options=opcoes,
        value=opcoes[0],
        description='Seção:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='400px')
    )
    
    return dropdown

# Criar widget de navegação
nav_widget = create_navigation_widget()
display(nav_widget)


In [ ]:
# Função para mostrar visão geral 2024
def show_overview_2024(dados):
    """Mostra visão geral dos dados de 2024"""
    
    if dados is None:
        print("❌ Dados não carregados")
        return
    
    print("🏠 VISÃO GERAL 2024 - DASHBOARD IMUNOPREVINIVEIS")
    print("=" * 60)
    
    # Dados gerais 2024
    if 'dados_gerais_2024' in dados:
        df_geral = dados['dados_gerais_2024']
        print(f"\n📊 Dados Gerais 2024:")
        print(df_geral.head())
        
        # Gráfico de casos por UF
        if 'UF' in df_geral.columns and 'Casos' in df_geral.columns:
            fig = px.bar(df_geral.head(10), x='UF', y='Casos', 
                        title='Top 10 UFs com mais casos em 2024')
            fig.show()
    
    # Dados de sorogrupos 2024
    if 'sorogrupos_2024' in dados:
        df_soro = dados['sorogrupos_2024']
        print(f"\n🦠 Sorogrupos 2024:")
        print(df_soro.head())
        
        # Gráfico de sorogrupos
        if 'Sorogrupo' in df_soro.columns and 'Casos' in df_soro.columns:
            fig = px.pie(df_soro, values='Casos', names='Sorogrupo', 
                        title='Distribuição de Sorogrupos em 2024')
            fig.show()
    
    # Dados de etiologia 2024
    if 'etiologia_2024' in dados:
        df_etio = dados['etiologia_2024']
        print(f"\n🔬 Etiologia 2024:")
        print(df_etio.head())
        
        # Gráfico de etiologia
        if 'Etiologia' in df_etio.columns and 'Casos' in df_etio.columns:
            fig = px.bar(df_etio, x='Etiologia', y='Casos', 
                        title='Casos por Etiologia em 2024')
            fig.show()
    
    print(f"\n📅 Última atualização: {datetime.now().strftime('%d/%m/%Y %H:%M')}")


In [ ]:
# Função principal de navegação
def on_navigation_change(change):
    """Função chamada quando a navegação muda"""
    
    clear_output(wait=True)
    
    # Recriar o widget de navegação
    display(nav_widget)
    
    opcao = change['new']
    
    if opcao == "🏠 Visão Geral 2024":
        show_overview_2024(dados)
    else:
        print(f"🔧 Seção '{opcao}' em desenvolvimento...")
        print("\n📋 Seções disponíveis:")
        print("- 🏠 Visão Geral 2024")
        print("\n💡 Para implementar outras seções, adapte as funções dos módulos app_sections/")

# Conectar o evento de mudança
nav_widget.observe(on_navigation_change, names='value')

print("🎯 Dashboard IMUNOPREVINIVEIS - Versão Jupyter Notebook")
print("📱 Use o menu dropdown acima para navegar entre as seções")
print("\n🚀 Seções implementadas:")
print("- 🏠 Visão Geral 2024")
print("\n💡 Outras seções podem ser implementadas adaptando as funções dos módulos app_sections/")


## 📚 Como Usar Este Notebook

1. **Execute as células em ordem** - Comece pela primeira célula e execute sequencialmente
2. **Use o menu dropdown** - Selecione a seção desejada no widget de navegação
3. **Visualize os gráficos** - Os gráficos Plotly são interativos
4. **Explore os dados** - Cada seção mostra diferentes análises dos dados

## 🔧 Personalização

Para adicionar mais seções, você pode:

1. **Importar funções dos módulos app_sections/**
2. **Adaptar as funções Streamlit** para funcionar no Jupyter
3. **Substituir `st.` por equivalentes do Jupyter** (print, display, widgets)

## 📦 Dependências Necessárias

```bash
pip install pandas numpy plotly matplotlib seaborn scipy scikit-learn ipywidgets
```

## 🎯 Vantagens da Versão Jupyter

- ✅ **Interativo** - Execute células individualmente
- ✅ **Reproduzível** - Salve e compartilhe o notebook
- ✅ **Flexível** - Modifique análises facilmente
- ✅ **Colaborativo** - Ideal para pesquisa e desenvolvimento
- ✅ **Visualizações inline** - Gráficos integrados ao notebook
